# Генерация датасетов моделью ChatGPT 4.1

In [1]:
EXAMPLE_ID = 0
MODEL = 'gpt-4.1'
START = 1000
OUT_COUNT = 250
OUT_FILE_PREFIX = 'chatgpt_essays'
PAUSE_SECONDS = 1

### Исходный датасет, написанный человеком

In [2]:
import pandas as pd

df = pd.read_csv("../datasets/human_essays.csv", encoding="utf-8")
df.head(10)

,title,text
0,12 Years a Slave: An Analysis of the Film Essay,The 2013 film 12 Years a Slave proved that sla...
1,20+ Social Media Post Ideas to Radically Simpl...,Social Media Examiner’s (2021) video on social...
2,533 U.S. 27 (2001) Kyllo v. United States: The...,Table of Contents\n 1. Facts\n 2. Issue\n 3. H...
3,A Charles Schwab Corporation Case Essay,Charles Schwab is a for-profit Corporation who...
4,A Clinical Office Assistant’s Attire Research ...,The work of a clinical or medical office worke...
5,A Comic Science Fiction Film “Back to the Futu...,Table of Contents\n 1. Introduction\n 2. Movie...
6,A Community Yard Sale as a Memorable Event Essay,Autumn is not generally viewed as an appropria...
7,A Complex in the “Every Secret Thing” Film by ...,Carl Jung identifies the mother complex with a...
8,A Customer Told to Wear a Mask Threw Snow at a...,The article describes a recent Chicago hot dog...
9,A Disconnect Between Public Transportation and...,One realistic way to commute is using private ...


#### Пример:

In [3]:
print(f"{df.iloc[EXAMPLE_ID,0]}\n\n{df.iloc[EXAMPLE_ID,1]}")

12 Years a Slave: An Analysis of the Film Essay

The 2013 film 12 Years a Slave proved that slavery is a worldwide issue. Indeed, the film made $150 million outside the United States and $57 million in the U.S., with a production budget of $20 million (Sharf, 2020). The movie was based on the memoir Twelve Years a Slave by Solomon Northup (Ntim, 2020). It tells the story of a free African American man who was kidnapped and sold into slavery. Solomon spent twelve years away from his family, being traded from one master to another. Fortunately, the protagonist met a person who helped him deliver a message to his family and friends, who came and rescued him. This movie accurately illustrates discriminatory relationships between white slaveholders and black slaves that stemmed from the dysfunctional system in the country and prejudices in people’s mindsets at that time.

The two main ethnic groups presented in this film are White and African Americans, and the three social groups are afflu

### Утилиты

In [4]:
import math
import re


def round_word_count(c):
    return math.floor(c / 100) * 100

def word_count(text):
    words = re.findall(r'\b\w+\b', text)
    return round_word_count(len(words))

def print_example(data, example_id):
    if 'prompt' in data.iloc[example_id]:
        print(f"Prompt {'=' * 93}\n{data.iloc[example_id]['prompt']}")
        print(f"{'=' * 100}")
        print(f"{data.iloc[example_id]['title']}\n\n{data.iloc[example_id]['text']}")
        print(f"{'=' * 100}")
    if 'prompt_2' in data.iloc[example_id]:
        print(f"Prompt 1 {'=' * 93}\n{data.iloc[example_id]['prompt_1']}")
        print(f"{'=' * 100}")
        print(f"{data.iloc[example_id]['response']}")
        print(f"Prompt 2 {'=' * 93}\n{data.iloc[example_id]['prompt_2']}")
        print(f"{'=' * 100}")
        print(f"{data.iloc[example_id]['title']}\n\n{data.iloc[example_id]['text']}")
        print(f"{'=' * 100}")

def print_progress(start, i, n):
    if (i - start) % (n / 100) == 0:
        print(f"{100 * (i - start) / n}% done")

In [5]:
from dotenv import load_dotenv
import os
from openai import OpenAI


load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
import time


def generate_response(prompt):
    response = client.responses.create(
        model=MODEL,
        input=prompt
    )

    time.sleep(PAUSE_SECONDS)
    
    return response.output_text

### Вариант 1. Prompt-based generation

```
Write an essay of about {} words on the topic '{}'.
Start directly with the first sentence of the essay. Absolutely no title or heading of any kind. 
Output only the essay text.
```

In [7]:
def generate(model_name, df, prompt_raw, start=0, n=5000):
    rows = []
    for i in range(start, start + n):
        title = df.loc[i, "title"]
        length = word_count(df.loc[i, "text"])
    
        prompt_formatted = prompt_raw.format(length, title)

        try:
            response = generate_response(prompt_formatted)
        except:
            continue
            
        response = response.lstrip()
    
        rows.append({"title": title, "prompt": prompt_formatted, "text": response, "model": model_name})

        print_progress(start, i, n)
    
    return pd.DataFrame(rows)

In [8]:
prompt = """
Write an essay of about {} words on the topic '{}'.
Start directly with the first sentence of the essay. Absolutely no title or heading of any kind. 
Output only the essay text. 
"""

df1 = generate(MODEL, df, prompt, start=START, n=OUT_COUNT)
df1.head()

0.0% done
2.0% done
4.0% done
6.0% done
8.0% done
10.0% done
12.0% done
14.0% done
16.0% done
18.0% done
20.0% done
22.0% done
24.0% done
26.0% done
28.0% done
30.0% done
32.0% done
34.0% done
36.0% done
38.0% done
40.0% done
42.0% done
44.0% done
46.0% done
48.0% done
50.0% done
52.0% done
54.0% done
56.0% done
58.0% done
60.0% done
62.0% done
64.0% done
66.0% done
68.0% done
70.0% done
72.0% done
74.0% done
76.0% done
78.0% done
80.0% done
82.0% done
84.0% done
86.0% done
88.0% done
90.0% done
92.0% done
94.0% done
96.0% done
98.0% done


,title,prompt,text,model
0,A Growing Market: Restructuring P&G Research P...,\nWrite an essay of about 600 words on the top...,"Procter & Gamble (P&G), a global leader in con...",gpt-4.1
1,A Manager’s Power Bases in a Healthcare Settin...,\nWrite an essay of about 600 words on the top...,A manager’s power bases in a healthcare settin...,gpt-4.1
2,A Maslenitsa Festival as a Cultural Event Essay,\nWrite an essay of about 1300 words on the to...,"Maslenitsa, also known as Butter Week or Panca...",gpt-4.1
3,A Modern Approach to Phineas Taylor Barnum’s B...,\nWrite an essay of about 600 words on the top...,"Phineas Taylor Barnum, the legendary 19th-cent...",gpt-4.1
4,A Personal Crisis that Prompted Me to Apply fo...,\nWrite an essay of about 600 words on the top...,When I first moved away from home to attend un...,gpt-4.1


#### Пример:

In [9]:
print_example(df1, EXAMPLE_ID)

Prompt =============================================================================================

Write an essay of about 600 words on the topic 'A Growing Market: Restructuring P&G Research Paper'.
Start directly with the first sentence of the essay. Absolutely no title or heading of any kind. 
Output only the essay text. 

A Growing Market: Restructuring P&G Research Paper

Procter & Gamble (P&G), a global leader in consumer goods, has exemplified adaptability and innovation throughout its history, allowing it to maintain a competitive edge in an ever-evolving market landscape. In recent years, the company has embarked on a significant restructuring of its research and development (R&D) strategies to address the challenges and opportunities presented by a growing and increasingly complex market. This essay examines the patterns and rationale behind P&G’s restructuring efforts, highlighting how they have served to strengthen the company’s position as an industry pioneer while assu

In [10]:
file_path = f"../datasets/{OUT_FILE_PREFIX}_1.csv"

df1.to_csv(
    file_path, 
    mode="a",
    header=not os.path.exists(file_path),
    index=False,
    encoding="utf-8",
)

### Вариант 2. Prompt-based Style Transfer

```
Write an essay of about {} words on the topic '{}', following the stylistic and structural conventions of IvyPanda essays.

STYLE REQUIREMENTS (IvyPanda-like):
1. The tone must be academic, clear, neutral, and objective.
2. The essay should include background context, a clear analytical structure, and well-developed paragraphs.
3. Arguments must be supported with reasoning, concise explanations, and relevant examples.
4. The writing must be coherent, logically progressive, and easy to read.
5. Avoid emotional language or overly dramatic phrasing.

STRUCTURE REQUIREMENTS:
1. Divide the essay into several sections, each starting with a plain-text subheading.
2. Subheadings must contain only letters and spaces — no punctuation, no symbols, and absolutely no Markdown formatting.
   - This means: no asterisks, no bold, no italics, no underscores, no brackets.
   - Subheadings must appear as plain text lines, e.g. Background Context
3. Each subheading must be on its own line with exactly one blank line before and after it.
4. No overall essay title — begin directly with the first subheading.
5. Output must be plain text only (no Markdown, no asterisks, no bold, no italics, no hashtags, no special formatting).

OPTIONAL ELEMENTS:
You may include a plain-text references section at the end if appropriate to the topic, but this is optional.

Begin with the first subheading, then continue with the first paragraph.
Output only the essay text.
```

In [11]:
def generate(model_name, df, prompt_raw, start=0, n=5000):
    rows = []
    for i in range(start, start + n):
        title = df.loc[i, "title"]
        length = word_count(df.loc[i, "text"])
    
        prompt_formatted = prompt_raw.format(length, title)
    
        try:
            response = generate_response(prompt_formatted)
        except:
            continue
            
        response = response.lstrip()
    
        rows.append({"title": title, "prompt": prompt_formatted, "text": response, "model": model_name})

        print_progress(start, i, n)
    
    return pd.DataFrame(rows)

In [12]:
prompt = """
Write an essay of about {} words on the topic '{}', following the stylistic and structural conventions of IvyPanda essays.

STYLE REQUIREMENTS (IvyPanda-like):
1. The tone must be academic, clear, neutral, and objective.
2. The essay should include background context, a clear analytical structure, and well-developed paragraphs.
3. Arguments must be supported with reasoning, concise explanations, and relevant examples.
4. The writing must be coherent, logically progressive, and easy to read.
5. Avoid emotional language or overly dramatic phrasing.

STRUCTURE REQUIREMENTS:
1. Divide the essay into several sections, each starting with a plain-text subheading.
2. Subheadings must contain only letters and spaces — no punctuation, no symbols, and absolutely no Markdown formatting.
   - This means: no asterisks, no bold, no italics, no underscores, no brackets.
   - Subheadings must appear as plain text lines, e.g. Background Context
3. Each subheading must be on its own line with exactly one blank line before and after it.
4. No overall essay title — begin directly with the first subheading.
5. Output must be plain text only (no Markdown, no asterisks, no bold, no italics, no hashtags, no special formatting).

OPTIONAL ELEMENTS:
You may include a plain-text references section at the end if appropriate to the topic, but this is optional.

Begin with the first subheading, then continue with the first paragraph.
Output only the essay text.
"""

df2 = generate(MODEL, df, prompt, start=START, n=OUT_COUNT)
df2.head()

0.0% done
2.0% done
4.0% done
6.0% done
8.0% done
10.0% done
12.0% done
14.0% done
16.0% done
18.0% done
20.0% done
22.0% done
24.0% done
26.0% done
28.0% done
30.0% done
32.0% done
34.0% done
36.0% done
38.0% done
40.0% done
42.0% done
44.0% done
46.0% done
48.0% done
50.0% done
52.0% done
54.0% done
56.0% done
58.0% done
60.0% done
62.0% done
64.0% done
66.0% done
68.0% done
70.0% done
72.0% done
74.0% done
76.0% done
78.0% done
80.0% done
82.0% done
84.0% done
86.0% done
88.0% done
90.0% done
92.0% done
94.0% done
96.0% done
98.0% done


,title,prompt,text,model
0,A Growing Market: Restructuring P&G Research P...,\nWrite an essay of about 600 words on the top...,Background Context\n\nProcter & Gamble (P&G) s...,gpt-4.1
1,A Manager’s Power Bases in a Healthcare Settin...,\nWrite an essay of about 600 words on the top...,Background Context\n\nIn modern healthcare org...,gpt-4.1
2,A Maslenitsa Festival as a Cultural Event Essay,\nWrite an essay of about 1300 words on the to...,Background Context\n\nMaslenitsa is a vibrant ...,gpt-4.1
3,A Modern Approach to Phineas Taylor Barnum’s B...,\nWrite an essay of about 600 words on the top...,"Background Context\n\nPhineas Taylor Barnum, b...",gpt-4.1
4,A Personal Crisis that Prompted Me to Apply fo...,\nWrite an essay of about 600 words on the top...,Background Context\n\nEmergency assistance pro...,gpt-4.1


#### Пример:

In [13]:
print_example(df2, EXAMPLE_ID)

Prompt =============================================================================================

Write an essay of about 600 words on the topic 'A Growing Market: Restructuring P&G Research Paper', following the stylistic and structural conventions of IvyPanda essays.

STYLE REQUIREMENTS (IvyPanda-like):
1. The tone must be academic, clear, neutral, and objective.
2. The essay should include background context, a clear analytical structure, and well-developed paragraphs.
3. Arguments must be supported with reasoning, concise explanations, and relevant examples.
4. The writing must be coherent, logically progressive, and easy to read.
5. Avoid emotional language or overly dramatic phrasing.

STRUCTURE REQUIREMENTS:
1. Divide the essay into several sections, each starting with a plain-text subheading.
2. Subheadings must contain only letters and spaces — no punctuation, no symbols, and absolutely no Markdown formatting.
   - This means: no asterisks, no bold, no italics, no undersco

In [14]:
file_path = f"../datasets/{OUT_FILE_PREFIX}_2.csv"

df2.to_csv(
    file_path, 
    mode="a",
    header=not os.path.exists(file_path),
    index=False,
    encoding="utf-8",
)

### Вариант 3. Few-shot prompting

```
You are a helpful writing assistant. Your task is to generate essays of a given length and topic.
Start directly with the first sentence. No titles or headings. Output only the essay text.

Here is an example:

Example topic: "{}"
Example length: {} words

Example output:
{}

Now generate a new essay.

Topic: "{}"
Length: {} words

Write the essay now.
```

В качестве примера используется эссе из этого же набора, но на другую тему.

In [15]:
def generate_few_shot(model_name, df, prompt_raw, start=0, n=5000):
    rows = []
    for i in range(start + 1, start + n + 1):
        prev_title = df.loc[i - 1, "title"]
        prev_length = word_count(df.loc[i - 1, "text"])
        prev_text = df.loc[i - 1, "text"]
        
        title = df.loc[i, "title"]
        length = word_count(df.loc[i, "text"])
    
        prompt_formatted = prompt_raw.format(prev_title, prev_length, prev_text, title, length)

        try:
            response = generate_response(prompt_formatted)
        except:
            continue
            
        response = response.lstrip()
    
        rows.append({"title": title, "prompt": prompt_formatted, "text": response, "model": model_name})

        print_progress(start, i, n)
    
    return pd.DataFrame(rows)

In [16]:
prompt = """
You are a helpful writing assistant. Your task is to generate essays of a given length and topic.
Start directly with the first sentence. No titles or headings. Output only the essay text.

Here is an example:

Example topic: "{}"
Example length: {} words

Example output:
{}

Now generate a new essay.

Topic: "{}"
Length: {} words

Write the essay now.
"""

df3 = generate_few_shot(MODEL, df, prompt, start=START, n=OUT_COUNT)
df3.head()

2.0% done
4.0% done
6.0% done
8.0% done
10.0% done
12.0% done
14.0% done
16.0% done
18.0% done
20.0% done
22.0% done
24.0% done
26.0% done
28.0% done
30.0% done
32.0% done
34.0% done
36.0% done
38.0% done
40.0% done
42.0% done
44.0% done
46.0% done
48.0% done
50.0% done
52.0% done
54.0% done
56.0% done
58.0% done
60.0% done
62.0% done
64.0% done
66.0% done
68.0% done
70.0% done
72.0% done
74.0% done
76.0% done
78.0% done
80.0% done
82.0% done
84.0% done
86.0% done
88.0% done
90.0% done
92.0% done
94.0% done
96.0% done
98.0% done
100.0% done


,title,prompt,text,model
0,A Manager’s Power Bases in a Healthcare Settin...,\nYou are a helpful writing assistant. Your ta...,"Power is a central concept in management, shap...",gpt-4.1
1,A Maslenitsa Festival as a Cultural Event Essay,\nYou are a helpful writing assistant. Your ta...,"Maslenitsa, also known as Butter Week or Panca...",gpt-4.1
2,A Modern Approach to Phineas Taylor Barnum’s B...,\nYou are a helpful writing assistant. Your ta...,"Phineas Taylor Barnum, often remembered as the...",gpt-4.1
3,A Personal Crisis that Prompted Me to Apply fo...,\nYou are a helpful writing assistant. Your ta...,"In the winter of my sophomore year of college,...",gpt-4.1
4,A Risk Analysis: Cultural Differences in a Mul...,\nYou are a helpful writing assistant. Your ta...,"In today’s interconnected world, multinational...",gpt-4.1


#### Пример:

In [17]:
print_example(df3, EXAMPLE_ID)

Prompt =============================================================================================

You are a helpful writing assistant. Your task is to generate essays of a given length and topic.
Start directly with the first sentence. No titles or headings. Output only the essay text.

Here is an example:

Example topic: "A Growing Market: Restructuring P&G Research Paper"
Example length: 600 words

Example output:
The business world is ever-changing, and companies must always be flexible to these changes. One of the changes that a business might need to adapt to is an expanding target market. P&G currently sells personal care products to a market that is comprised primarily of women. However, research shows that there is a growing market among male consumers due to the rise of male grooming. P&G should restructure to take advantage of the new market for men’s care products.

P&G will need to make several changes to capitalize on the “ignored” market. The restructuring will affect

In [18]:
file_path = f"../datasets/{OUT_FILE_PREFIX}_3.csv"

df3.to_csv(
    file_path, 
    mode="a",
    header=not os.path.exists(file_path),
    index=False,
    encoding="utf-8",
)

### Вариант 4. Content Abstraction (Regeneration Learning)

Первый промпт:
```
Identify the writing style and format of the text, excluding all semantic content and themes.

Text:
{}
```

Второй промпт:
```
Write an essay of about {} words on the topic '{}'.

You are given the following style-and-format description:
'{}'

Start directly with the first sentence of the essay. Absolutely no title or heading of any kind.
Output only the essay text.
```

In [19]:
def generate_with_content(model_name, df, prompt_raw_1, prompt_raw_2, start=0, n=5000):
    rows = []
    for i in range(start, start + n):
        title = df.loc[i, "title"]
        length = word_count(df.loc[i, "text"])
        text = df.loc[i, "text"]

        prompt_1_formatted = prompt_raw_1.format(text)

        try:
            response_1 = generate_response(prompt_1_formatted)
            response_1 = response_1.lstrip()
    
            prompt_2_formatted = prompt_raw_2.format(length, title, response_1)
            response_2 = generate_response(prompt_2_formatted)
        except:
            continue
            
        response = response_2.lstrip()
    
        rows.append({
            "title": title, 
            "prompt_1": prompt_1_formatted, 
            "response": response_1,
            "prompt_2": prompt_2_formatted, 
            "text": response, 
            "model": model_name
        })

        print_progress(start, i, n)
    
    return pd.DataFrame(rows)

In [20]:
prompt_1 = """
Identify the writing style and format of the text, excluding all semantic content and themes.

Text:
{}
"""

prompt_2 = """
Write an essay of about {} words on the topic '{}'.

You are given the following style-and-format description:
'{}'

Start directly with the first sentence of the essay. Absolutely no title or heading of any kind.
Output only the essay text.
"""

In [21]:
df4 = generate_with_content(MODEL, df, prompt_1, prompt_2, start=START, n=OUT_COUNT)
df4.head()

0.0% done
2.0% done
4.0% done
6.0% done
8.0% done
10.0% done
12.0% done
14.0% done
16.0% done
18.0% done
20.0% done
22.0% done
24.0% done
26.0% done
28.0% done
30.0% done
32.0% done
34.0% done
36.0% done
38.0% done
40.0% done
42.0% done
44.0% done
46.0% done
48.0% done
50.0% done
52.0% done
54.0% done
56.0% done
58.0% done
60.0% done
62.0% done
64.0% done
66.0% done
68.0% done
70.0% done
72.0% done
74.0% done
76.0% done
78.0% done
80.0% done
82.0% done
84.0% done
86.0% done
88.0% done
90.0% done
92.0% done
94.0% done
96.0% done
98.0% done


,title,prompt_1,response,prompt_2,text,model
0,A Growing Market: Restructuring P&G Research P...,\nIdentify the writing style and format of the...,**Writing Style:** \nThe text is written in a...,\nWrite an essay of about 600 words on the top...,The global market landscape is characterized b...,gpt-4.1
1,A Manager’s Power Bases in a Healthcare Settin...,\nIdentify the writing style and format of the...,**Writing Style:** \nThe text employs a forma...,\nWrite an essay of about 600 words on the top...,In the complex environment of the healthcare s...,gpt-4.1
2,A Maslenitsa Festival as a Cultural Event Essay,\nIdentify the writing style and format of the...,**Writing Style:** \n- Formal and academic to...,\nWrite an essay of about 1300 words on the to...,"Maslenitsa, also known as Butter Week or Panca...",gpt-4.1
3,A Modern Approach to Phineas Taylor Barnum’s B...,\nIdentify the writing style and format of the...,Certainly! Here is an analysis of the **writin...,\nWrite an essay of about 600 words on the top...,"Phineas Taylor Barnum, a renowned figure in ni...",gpt-4.1
4,A Personal Crisis that Prompted Me to Apply fo...,\nIdentify the writing style and format of the...,**Writing Style and Format Analysis (Excluding...,\nWrite an essay of about 600 words on the top...,"Emergencies, by their nature, are unexpected s...",gpt-4.1


#### Пример:

In [22]:
print_example(df4, EXAMPLE_ID)

Prompt 1 =============================================================================================

Identify the writing style and format of the text, excluding all semantic content and themes.

Text:
The business world is ever-changing, and companies must always be flexible to these changes. One of the changes that a business might need to adapt to is an expanding target market. P&G currently sells personal care products to a market that is comprised primarily of women. However, research shows that there is a growing market among male consumers due to the rise of male grooming. P&G should restructure to take advantage of the new market for men’s care products.

P&G will need to make several changes to capitalize on the “ignored” market. The restructuring will affect various departments such as research and development, product design, and sales and marketing. Their redesign will affect their employee recruitment and selection practices (Szymczyk, 2016). For instance, currently, mo

In [23]:
file_path = f"../datasets/{OUT_FILE_PREFIX}_4.csv"

df4.to_csv(
    file_path, 
    mode="a",
    header=not os.path.exists(file_path),
    index=False,
    encoding="utf-8",
)

### Вариант 5. Outline‑guided Text Generation

Первый промпт:
```
You are an academic writing instructor.
Create a well-structured outline for an academic essay on the following topic: {}.

Requirements:

The outline must include the following sections:
- Introduction
- Main Body (2–4 thematic sections)
- Counterarguments / Alternative Perspectives
- Conclusion

Each section should contain 2–3 concise bullet points describing the key ideas or arguments.
The outline should reflect a clear thesis-driven structure.
Use formal academic language.

Do not write the essay itself
```

Второй промпт:
```
You are an academic essay writer.
Write a coherent academic essay of about {} words on the topic '{}' strictly following the outline below.

Outline:
{}

Requirements:
- Follow the outline in the given order without adding or removing sections.
- Develop each bullet point into a full, well-argued paragraph.
- Maintain a formal academic tone and clear argumentative logic.
- Use appropriate academic transitions and signposting.
- Do not introduce new arguments or themes not present in the outline.
- Avoid informal language, personal anecdotes, and unsupported claims.
```

In [24]:
def generate_with_plan(model_name, df, prompt_raw_1, prompt_raw_2, start=0, n=5000):
    rows = []
    for i in range(start, start + n):
        title = df.loc[i, "title"]
        length = word_count(df.loc[i, "text"])
        text = df.loc[i, "text"]
    
        prompt_1_formatted = prompt_raw_1.format(title)

        try:
            response_1 = generate_response(prompt_1_formatted)
            response_1 = response_1.lstrip()
    
            prompt_2_formatted = prompt_raw_2.format(length, title, response_1)
            response_2 = generate_response(prompt_2_formatted)
        except:
            continue
            
        response = response_2.lstrip()
    
        rows.append({
            "title": title, 
            "prompt_1": prompt_1_formatted, 
            "response": response_1,
            "prompt_2": prompt_2_formatted, 
            "text": response, 
            "model": model_name
        })

        print_progress(start, i, n)
    
    return pd.DataFrame(rows)

In [25]:
prompt_1 = """
You are an academic writing instructor.
Create a well-structured outline for an academic essay on the following topic: {}.

Requirements:

The outline must include the following sections:
- Introduction
- Main Body (2–4 thematic sections)
- Counterarguments / Alternative Perspectives
- Conclusion

Each section should contain 2–3 concise bullet points describing the key ideas or arguments.
The outline should reflect a clear thesis-driven structure.
Use formal academic language.

Do not write the essay itself
"""

prompt_2 = """
You are an academic essay writer.
Write a coherent academic essay of about {} words on the topic '{}' strictly following the outline below.

Outline:
{}

Requirements:
- Follow the outline in the given order without adding or removing sections.
- Develop each bullet point into a full, well-argued paragraph.
- Maintain a formal academic tone and clear argumentative logic.
- Use appropriate academic transitions and signposting.
- Do not introduce new arguments or themes not present in the outline.
- Avoid informal language, personal anecdotes, and unsupported claims.
"""

In [26]:
df5 = generate_with_plan(MODEL, df, prompt_1, prompt_2, start=START, n=OUT_COUNT)
df5.head()

0.0% done
2.0% done
4.0% done
6.0% done
8.0% done
10.0% done
12.0% done
14.0% done
16.0% done
18.0% done
20.0% done
22.0% done
24.0% done
26.0% done
28.0% done
30.0% done
32.0% done
34.0% done
36.0% done
38.0% done
40.0% done
42.0% done
44.0% done
46.0% done
48.0% done
50.0% done
52.0% done
54.0% done
56.0% done
58.0% done
60.0% done
62.0% done
64.0% done
66.0% done
68.0% done
70.0% done
72.0% done
74.0% done
76.0% done
78.0% done
80.0% done
82.0% done
84.0% done
86.0% done
88.0% done
90.0% done
92.0% done
94.0% done
96.0% done
98.0% done


,title,prompt_1,response,prompt_2,text,model
0,A Growing Market: Restructuring P&G Research P...,\nYou are an academic writing instructor.\nCre...,**Outline: A Growing Market: Restructuring P&G...,\nYou are an academic essay writer.\nWrite a c...,**A Growing Market: Restructuring P&G Research...,gpt-4.1
1,A Manager’s Power Bases in a Healthcare Settin...,\nYou are an academic writing instructor.\nCre...,**Outline: A Manager’s Power Bases in a Health...,\nYou are an academic essay writer.\nWrite a c...,**A Manager’s Power Bases in a Healthcare Sett...,gpt-4.1
2,A Maslenitsa Festival as a Cultural Event Essay,\nYou are an academic writing instructor.\nCre...,**Outline: A Maslenitsa Festival as a Cultural...,\nYou are an academic essay writer.\nWrite a c...,**A Maslenitsa Festival as a Cultural Event**\...,gpt-4.1
3,A Modern Approach to Phineas Taylor Barnum’s B...,\nYou are an academic writing instructor.\nCre...,**Outline: A Modern Approach to Phineas Taylor...,\nYou are an academic essay writer.\nWrite a c...,**A Modern Approach to Phineas Taylor Barnum’s...,gpt-4.1
4,A Personal Crisis that Prompted Me to Apply fo...,\nYou are an academic writing instructor.\nCre...,**Outline: A Personal Crisis that Prompted Me ...,\nYou are an academic essay writer.\nWrite a c...,**A Personal Crisis that Prompted Me to Apply ...,gpt-4.1


#### Пример:

In [27]:
print_example(df5, EXAMPLE_ID)

Prompt 1 =============================================================================================

You are an academic writing instructor.
Create a well-structured outline for an academic essay on the following topic: A Growing Market: Restructuring P&G Research Paper.

Requirements:

The outline must include the following sections:
- Introduction
- Main Body (2–4 thematic sections)
- Counterarguments / Alternative Perspectives
- Conclusion

Each section should contain 2–3 concise bullet points describing the key ideas or arguments.
The outline should reflect a clear thesis-driven structure.
Use formal academic language.

Do not write the essay itself

**Outline: A Growing Market: Restructuring P&G Research Paper**

**I. Introduction**
- Introduce the significance of Procter & Gamble (P&G) as a leading multinational corporation in the consumer goods industry.
- Present the context of recent market growth and the need for organizational restructuring to remain competitive.
- Thesis

In [28]:
file_path = f"../datasets/{OUT_FILE_PREFIX}_5.csv"

df5.to_csv(
    file_path, 
    mode="a",
    header=not os.path.exists(file_path),
    index=False,
    encoding="utf-8",
)